### Import dependencies

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Merge datasets

In [2]:
# Read in data
df = pd.read_csv("resources/all.csv")

In [3]:
# Sort data frame by quarter
df = df.sort_values(by=['quarter'])

In [4]:
# Set index to quarter
df = df.set_index('quarter')

In [5]:
# Rename target column
df = df.rename(columns={'target':'recession_actual'})
df.shape

(171, 10)

### Shift data with sliding window technique

In [6]:
df['recession_1q_out'] = df['recession_actual'].shift(-1)
df['recession_2q_out'] = df['recession_actual'].shift(-2)
df['recession_4q_out'] = df['recession_actual'].shift(-4)

In [7]:
# Create three datasets -- 1 for each model (recession 1Qtr out, 2Qtrs out, 4Qtrs out)
df_q1 = df.drop(columns=['recession_2q_out','recession_4q_out','recession_actual'])
df_q2 = df.drop(columns=['recession_4q_out','recession_1q_out','recession_actual'])
df_q4 = df.drop(columns=['recession_1q_out','recession_2q_out','recession_actual'])

In [8]:
# Delete missing values
df_q1 = df_q1.dropna()
df_q2 = df_q2.dropna()
df_q4 = df_q4.dropna()

In [10]:
# Define y variables
y1 = df_q1['recession_1q_out']
y2 = df_q2['recession_2q_out']
y3 = df_q4['recession_4q_out']

In [11]:
# Drop target
df_q1 = df_q1.drop(columns=['recession_1q_out'])
df_q2 = df_q2.drop(columns=['recession_2q_out'])
df_q4 = df_q4.drop(columns=['recession_4q_out'])

In [12]:
# Define X
X_q1 = df_q1
X_q2 = df_q2
X_q4 = df_q4

### Split and scale data

In [13]:
# Split data into training and testing
X1_train, X1_test, y1_train, y1_test=train_test_split(X_q1, y1, train_size=0.8, random_state=42, stratify=y1)
X2_train, X2_test, y2_train, y2_test=train_test_split(X_q2, y2, train_size=0.8, random_state=42, stratify=y2)
X3_train, X3_test, y3_train, y3_test=train_test_split(X_q4, y3, train_size=0.8, random_state=42, stratify=y3)

In [14]:
# Create scaler object
X1_scaler = StandardScaler().fit(X1_train)
X2_scaler = StandardScaler().fit(X2_train)
X3_scaler = StandardScaler().fit(X3_train)

In [15]:
# Scale training data
X1_train_scaled = X1_scaler.transform(X1_train)
X2_train_scaled = X2_scaler.transform(X2_train)
X3_train_scaled = X3_scaler.transform(X3_train)

# Scale testing data
X1_test_scaled = X1_scaler.transform(X1_test)
X2_test_scaled = X2_scaler.transform(X2_test)
X3_test_scaled = X3_scaler.transform(X3_test)

### Reshape data to fit LSTM format

In [16]:
# Method to reshape data
def reshape_data(obj):
    reshaped_obj = np.reshape(obj, (obj.shape[0], obj.shape[1], 1))
    return reshaped_obj

In [17]:
# Reshape training data
reshaped_X1_train_scaled = reshape_data(X1_train_scaled)
reshaped_X2_train_scaled = reshape_data(X2_train_scaled)
reshaped_X3_train_scaled = reshape_data(X3_train_scaled)

In [18]:
# Reshape testing data
reshaped_X1_test_scaled = reshape_data(X1_test_scaled)
reshaped_X2_test_scaled = reshape_data(X2_test_scaled)
reshaped_X3_test_scaled = reshape_data(X3_test_scaled)

## Build Model

In [19]:
# Initialize model
model = Sequential()

In [20]:
# Add layers
model.add(LSTM(128, input_shape=(reshaped_X1_train_scaled.shape[1],1), return_sequences=True, recurrent_dropout=0.1))
model.add(Dropout(0.4))
model.add(BatchNormalization())  # Normalize activation outputs

model.add(LSTM(128, return_sequences=True, recurrent_dropout=0.1))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(2, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [21]:
# Compile model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

**Train and predict on X1-Y1 data (recession 1 quarter out)**

In [22]:
# Fit the model to the training data
# Shuffle True/False to randomize the training data rows being fed into the model
model.fit(reshaped_X1_train_scaled, y1_train, epochs=100, shuffle=True, verbose=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
136/136 - 9s - loss: 0.9697 - acc: 0.5074
Epoch 2/100
136/136 - 1s - loss: 0.7019 - acc: 0.5809
Epoch 3/100
136/136 - 1s - loss: 0.4255 - acc: 0.7794
Epoch 4/100
136/136 - 1s - loss: 0.5220 - acc: 0.7500
Epoch 5/100
136/136 - 1s - loss: 0.5222 - acc: 0.7868
Epoch 6/100
136/136 - 1s - loss: 0.4838 - acc: 0.8235
Epoch 7/100
136/136 - 1s - loss: 0.4483 - acc: 0.8235
Epoch 8/100
136/136 - 1s - loss: 0.4337 - acc: 0.8162
Epoch 9/100
136/136 - 1s - loss: 0.3927 - acc: 0.8456
Epoch 10/100
136/136 - 1s - loss: 0.4039 - acc: 0.8162
Epoch 11/100
136/136 - 1s - loss: 0.3026 - acc: 0.8162
Epoch 12/100
136/136 - 1s - loss: 0.3824 - acc: 0.8382
Epoch 13/100
136/136 - 1s - loss: 0.4277 - acc: 0.7868
Epoch 14/100
136/136 - 1s - loss: 0.4227 - acc: 0.8088
Epoch 15/100
136/136 - 1s - loss: 0.3977 - acc: 0.8309
Epoch 16/100
136/136 - 1s - loss: 0.4017 - acc: 0.7941
Epoch 17/100
136/136 - 1s - loss: 0

In [23]:
# Validate model using test data
model_loss1, model_accuracy1 = model.evaluate(reshaped_X1_test_scaled, y1_test, verbose=2)

34/34 - 2s - loss: 0.5015 - acc: 0.7353


In [29]:
# Make predictions using test data
predictions1 = model.predict_classes(reshaped_X1_test_scaled)
predictions1

array([0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0])

**Train and predict on X2-Y2 data (recession 2 quarters out)**

In [25]:
# Fit the model to the training data
model.fit(reshaped_X2_train_scaled, y2_train, epochs=100, shuffle=True, verbose=2)

Epoch 1/100
135/135 - 1s - loss: 0.2934 - acc: 0.8593
Epoch 2/100
135/135 - 1s - loss: 0.2631 - acc: 0.8889
Epoch 3/100
135/135 - 1s - loss: 0.2202 - acc: 0.8963
Epoch 4/100
135/135 - 1s - loss: 0.2378 - acc: 0.8963
Epoch 5/100
135/135 - 1s - loss: 0.2344 - acc: 0.9111
Epoch 6/100
135/135 - 1s - loss: 0.2368 - acc: 0.8889
Epoch 7/100
135/135 - 1s - loss: 0.2429 - acc: 0.8741
Epoch 8/100
135/135 - 1s - loss: 0.2030 - acc: 0.8815
Epoch 9/100
135/135 - 1s - loss: 0.2130 - acc: 0.8963
Epoch 10/100
135/135 - 1s - loss: 0.2328 - acc: 0.8889
Epoch 11/100
135/135 - 1s - loss: 0.2068 - acc: 0.9037
Epoch 12/100
135/135 - 1s - loss: 0.1773 - acc: 0.9259
Epoch 13/100
135/135 - 1s - loss: 0.1970 - acc: 0.9185
Epoch 14/100
135/135 - 1s - loss: 0.1857 - acc: 0.9111
Epoch 15/100
135/135 - 1s - loss: 0.1887 - acc: 0.9185
Epoch 16/100
135/135 - 1s - loss: 0.1948 - acc: 0.9037
Epoch 17/100
135/135 - 1s - loss: 0.2254 - acc: 0.8963
Epoch 18/100
135/135 - 1s - loss: 0.2013 - acc: 0.8741
Epoch 19/100
135/13

In [27]:
# Validate model using test data
model_loss2, model_accuracy2 = model.evaluate(reshaped_X2_test_scaled, y2_test, verbose=2)

34/34 - 0s - loss: 0.1907 - acc: 0.9412


In [30]:
# Make predictions using test data
predictions2 = model.predict_classes(reshaped_X2_test_scaled)
predictions2

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0])

**Train and predict on X3-Y3 data (recession 4 quarters out)**

In [31]:
# Fit the model to the training data
model.fit(reshaped_X3_train_scaled, y3_train, epochs=100, shuffle=True, verbose=2)

Epoch 1/100
133/133 - 1s - loss: 0.3635 - acc: 0.8722
Epoch 2/100
133/133 - 1s - loss: 0.2961 - acc: 0.8722
Epoch 3/100
133/133 - 1s - loss: 0.2129 - acc: 0.9023
Epoch 4/100
133/133 - 1s - loss: 0.2130 - acc: 0.9173
Epoch 5/100
133/133 - 1s - loss: 0.2176 - acc: 0.8872
Epoch 6/100
133/133 - 1s - loss: 0.1838 - acc: 0.9173
Epoch 7/100
133/133 - 1s - loss: 0.1620 - acc: 0.9173
Epoch 8/100
133/133 - 1s - loss: 0.1634 - acc: 0.9098
Epoch 9/100
133/133 - 1s - loss: 0.2026 - acc: 0.8947
Epoch 10/100
133/133 - 1s - loss: 0.2221 - acc: 0.8647
Epoch 11/100
133/133 - 1s - loss: 0.1815 - acc: 0.9098
Epoch 12/100
133/133 - 1s - loss: 0.1689 - acc: 0.9098
Epoch 13/100
133/133 - 1s - loss: 0.2610 - acc: 0.8872
Epoch 14/100
133/133 - 1s - loss: 0.1876 - acc: 0.9173
Epoch 15/100
133/133 - 1s - loss: 0.2291 - acc: 0.9098
Epoch 16/100
133/133 - 1s - loss: 0.1523 - acc: 0.9248
Epoch 17/100
133/133 - 1s - loss: 0.1813 - acc: 0.9098
Epoch 18/100
133/133 - 1s - loss: 0.1656 - acc: 0.9023
Epoch 19/100
133/13

In [32]:
# Validate model using test data
model_loss3, model_accuracy3 = model.evaluate(reshaped_X3_test_scaled, y3_test, verbose=2)

34/34 - 0s - loss: 0.1540 - acc: 0.9706


In [33]:
# Make predictions using test data
predictions3 = model.predict_classes(reshaped_X3_test_scaled)
predictions3

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0])